In [72]:
import pandas as pd

# Sample color data
colors_data = {
    'color': ['Red', 'Green', 'Blue', 'Yellow', 'Purple','Black','Orange','Pink','Gray'],
    'colorName': ['Red', 'Green', 'Blue', 'Yellow', 'Purple','Black','Orange','Pink','Gray'],  # Hex values
    'hexValue': ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#800080','#000000','#FFA500','#FFC0CB','#808080'],
   
    'R': [255, 0, 0, 255, 128, 0, 255, 255, 128],
    'G': [0, 255, 0, 255, 0, 0, 165, 192, 128],
    'B': [0, 0, 255, 0, 128, 0, 0, 203, 128]
}

# Create DataFrame
df = pd.DataFrame(colors_data)

# Save DataFrame to CSV file
df.to_csv('colorsDataset.csv', index=False, header=False)


In [73]:
import tkinter as tk
from tkinter import filedialog
import cv2
import pandas as pd
import pyttsx3

dataset = None  # Define dataset as a global variable

def select_image():
    try:
        file_path = filedialog.askopenfilename()
        print("File selected:", file_path)
    except Exception as e:
        print("An error occurred during file selection:", e)
        
    if file_path:
        image_path.set(file_path)
        image = cv2.imread(file_path)
        if image is not None:
            image = cv2.resize(image, (800, 600))
            cv2.imshow("Uploaded", image)
            cv2.setMouseCallback("Uploaded", lambda event, x, y, flags, param: draw_function(event, x, y, flags, param, image))  # Pass image to draw_function
            cv2.waitKey(0)  # Wait for a key press to close the window
            cv2.destroyAllWindows()
        else:
            print("Error: Unable to read the image file.")

def draw_function(event, x, y, flags, param, image):
    global red, blue, green
    if event == cv2.EVENT_LBUTTONDBLCLK:
        blue, green, red = image[y, x]  # Accessing pixel values
        print("Red:", red, "Green:", green, "Blue:", blue)  # Debugging statement
        color_name = get_color_name(red, green, blue)
        print("Color Name:", color_name)  # Debugging statement
        speak_color(color_name)

def get_color_name(red, green, blue):
    global dataset
    if dataset is None:
        print("Error: Dataset is not loaded.")
        return ""

    min_value = 10000
    color_name = None  # Initialize color_name variable
    for i in range(len(dataset)):
        try:
            min_difference = abs(red - dataset.loc[i, 'R']) + abs(blue - dataset.loc[i, 'B']) + abs(green - dataset.loc[i, 'G'])
            if min_difference <= min_value:
                min_value = min_difference
                color_name = dataset.loc[i, 'colorName']
        except Exception as e:
            print("Error processing dataset:", e)
    if color_name is None:
        print("Error: Unable to determine color name.")
        return ""
    return color_name

def speak_color(color_name):
    if color_name:
        engine = pyttsx3.init()
        engine.say(color_name)
        engine.runAndWait()
    else:
        print("Error: Cannot speak an empty color name.")

root = tk.Tk()
root.title("Color Identifier")

image_path = tk.StringVar()
image_path.set("")

label = tk.Label(root, text="Select Image:")
label.grid(row=0, column=0, padx=5, pady=5)

button = tk.Button(root, text="Browse", command=select_image)
button.grid(row=0, column=1, padx=5, pady=5)

image_label = tk.Label(root, textvariable=image_path)
image_label.grid(row=0, column=2, padx=5, pady=5)

try:
    data_path = 'colorsDataset.csv'
    col_names = ['color', 'colorName', 'hexValue', 'R', 'G', 'B']
    dataset = pd.read_csv(data_path, names=col_names, header=None, dtype={'R': int, 'G': int, 'B': int})
    print("Dataset loaded successfully.")
    if dataset.empty:
        raise ValueError("Dataset is empty")
except FileNotFoundError:
    print("Error: File not found. Please check the file path:", data_path)
except Exception as e:
    print("Error loading dataset:", e)

root.mainloop()


Dataset loaded successfully.
File selected: C:/Users/navya/Downloads/Rainbow.png
Red: 0 Green: 187 Blue: 0
Color Name: Green
Red: 254 Green: 246 Blue: 0
Color Name: Yellow
Red: 0 Green: 0 Blue: 0
Color Name: Black
Red: 0 Green: 155 Blue: 254
Color Name: Blue
